In [102]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import random

In [98]:

train_data = pd.read_csv(r'C:\Users\lscon\Desktop\AA\projeto\the-three-body-problem\mlNOVA\X_train.csv')
#test_data = pd.read_csv(r'C:/Users/duart/OneDrive/Ambiente_de_Trabalho/Master_Analysis_Engineering_Big_Data/23-24/1st_semester/AA_ML/Kaggle_challenges/3_body_problem/3_body_problem/X_test.csv')

#Definir as variáveis
t = pd.DataFrame(train_data, columns=['t'])
x1 = pd.DataFrame(train_data, columns=['x_1'])
x2 = pd.DataFrame(train_data, columns=['x_2'])
x3 = pd.DataFrame(train_data, columns=['x_3'])
y1 = pd.DataFrame(train_data, columns=['y_1'])
y2 = pd.DataFrame(train_data, columns=['y_2'])
y3 = pd.DataFrame(train_data, columns=['y_3'])
vx1 = pd.DataFrame(train_data, columns=['v_x_1'])
vx2 = pd.DataFrame(train_data, columns=['v_x_2'])
vx3 = pd.DataFrame(train_data, columns=['v_x_3'])
vy1 = pd.DataFrame(train_data, columns=['v_y_1'])
vy2 = pd.DataFrame(train_data, columns=['v_y_2'])
vy3 = pd.DataFrame(train_data, columns=['v_y_3'])

In [ ]:
# Identify faulty rows based on the criterion (all values = 0.0 except for Id)
zero_rows = train_data[(train_data.drop('Id', axis=1) == 0).all(axis=1)]

# Remove the faulty rows from the DataFrame
train_data_preprocessed = train_data[~train_data.index.isin(zero_rows.index)]
train_data_preprocessed.reset_index(drop=True, inplace=True)
#train_data_preprocessed.to_csv('train_preprocessed.csv', index=False)

In [ ]:
#gives the stats for the preprocessed data (without the rows with zeros)
summary_stats_filtered = train_data_preprocessed.describe(include='all')

#gives the stats for the nonprocessed data
summary_stats = train_data.describe(include="all")

# Test...

In [ ]:
# Calculate the correlation matrix and plot it as a heatmap
corr_matrix = train_data_preprocessed.drop(train_data_preprocessed.columns[13],
                                       axis=1).corr()
corr_matrix.to_excel('corr_matrix_train_processed.xlsx')

plt.figure(figsize=(20, 16), dpi=800)
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.4f',
            linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.savefig('Corr_matrix_heatmap.jpg', dpi=800)
plt.show()

# Create a pairwise scatter plot matrix
scatter_matrix = pd.plotting.scatter_matrix(train_data_preprocessed,
                                            figsize=(20, 20))
plt.show()


In [ ]:
#create pairwise plots of correlation between variables

rows = 50000
x1_rows = x1.head(n=rows)
partial_train_data = train_data_preprocessed.drop(columns=['v_x_1','v_x_2','v_y_1', 
                                                           'v_y_2', 'v_x_3', 'v_y_3']).head(n=rows)


_= sns.pairplot(partial_train_data, kind="reg", diag_kind="kde", plot_kws={'line_kws':{'color':'red'}})
plt.title('Pairwise plots t vs velocity components')
plt.savefig('pairwisetv_50000')

In [103]:
#create feature label matrices
#we're not going to use the velocity components as features
train_data_without_velocity = train_data_preprocessed.drop(columns=['Id','v_x_1','v_x_2','v_y_1', 
                                                           'v_y_2', 'v_x_3', 'v_y_3'])
#divide by simulations
list_of_times = [values for values in train_data_without_velocity['t']]
time_index_tuples = list(enumerate(list_of_times))
zeros_indexes = list(filter(lambda value: value[1] == 0, time_index_tuples))
zeros_indexes = [value[0] for value in zeros_indexes] 
list_of_simulations = []
lower_bound = 0
for i in range(len(zeros_indexes)-1):
    simulation = train_data_without_velocity.iloc[lower_bound:zeros_indexes[i+1]]
    list_of_simulations.append(simulation)
    lower_bound = zeros_indexes[i+1]
from tqdm import tqdm

#add label and put the starting position at every row
#x1
for simulation in tqdm(list_of_simulations):
    first_row_values = simulation.head(1)
    simulation.loc[:,'x_1_label'] = simulation.loc[:, 'x_1']
    for index, row in simulation.iterrows():
        simulation.at[index, 'x_1'] = first_row_values['x_1']
        simulation.at[index, 'y_1'] = first_row_values['y_1']
        simulation.at[index, 'x_2'] = first_row_values['x_2']
        simulation.at[index, 'y_2'] = first_row_values['y_2']
        simulation.at[index, 'x_3'] = first_row_values['x_3']
        simulation.at[index, 'y_3'] = first_row_values['y_3']

list_of_simulations_copy = list_of_simulations.copy()
random.shuffle(list_of_simulations_copy) #we shuffle the data here so we are only shuffling different simulations and not amongst them
all_simulations = pd.concat(list_of_simulations_copy, ignore_index=True)
all_simulations.to_csv('feature_matrice_x1.csv', index=False)


  0%|          | 0/4997 [00:00<?, ?it/s]C:\Users\lscon\AppData\Local\Temp\ipykernel_6400\91045580.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulation.loc[:,'x_1_label'] = simulation.loc[:, 'x_1']
  0%|          | 1/4997 [00:00<09:54,  8.40it/s]C:\Users\lscon\AppData\Local\Temp\ipykernel_6400\91045580.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulation.loc[:,'x_1_label'] = simulation.loc[:, 'x_1']
  0%|          | 2/4997 [00:00<09:04,  9.17it/s]C:\Users\lscon\AppData\Local\Temp\ipyker

In [130]:
#load the feature matrice file
total_data = np.genfromtxt('feature_matrice_x1.csv', delimiter=',')
total_data = total_data[1:] 


array([ 0.        ,  1.        ,  0.        , -0.19554275,  0.69353823,
       -0.80445725, -0.69353823,  1.        ])

In [127]:
#splitting the dataset into training and validation
data_train, data_vali = train_test_split(total_data, test_size=0.7, shuffle=False)

In [135]:
#split the labels from the features
features_train = data_train[:, :-1]  
labels_train = data_train[:, -1] 

features_vali = data_vali[:, :-1]  
labels_vali = data_vali[:, -1] 


array([[ 0.        ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.0390625 ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.078125  ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.117188  ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.15625   ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.195312  ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.234375  ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.273438  ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.3125    ,  1.        ,  0.        , -0.19554275,  0.69353823,
        -0.80445725, -0.69353823],
       [ 0.351562  ,  1.    

In [132]:
#standardizing the data using z-scoring
scaler = StandardScaler()
scaler.fit(features_train)
scaler.transform(features_train)
scaler.transform(features_train)

array([[-1.67740995,  0.        ,  0.        , ...,  0.75460013,
        -0.24960003, -0.75460013],
       [-1.66403195,  0.        ,  0.        , ...,  0.75460013,
        -0.24960003, -0.75460013],
       [-1.65065395,  0.        ,  0.        , ...,  0.75460013,
        -0.24960003, -0.75460013],
       ...,
       [-0.91486474,  0.        ,  0.        , ...,  0.17078355,
         0.37319749, -0.17078355],
       [-0.90148759,  0.        ,  0.        , ...,  0.17078355,
         0.37319749, -0.17078355],
       [-0.88810702,  0.        ,  0.        , ...,  0.17078355,
         0.37319749, -0.17078355]])

In [ ]:
#plot the points (isto e porque acho que era interessante termos no deck of slides)

In [ ]:
#polynomial features - let's check the best polynomial degree + ridge regression
#escolher com base no menor MSE com o VALIDATION SET
#entretanto descobri que o sklearn tem uma crossvalidation feature que e capaz de ser bem util no calculo do MSE (literalmente transformar
# aquilo em duas linhas o que e fixolas)
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score

In [ ]:
#depois no final grafico para o melhor claro e calculamos sqrt(mse) para vermos o quao off estamos - test set
#depois seria implementar isto tudo para as outras matrizes que faltam
#sugeria depois transformar isto do modelo numa funcao para ser mais simples e nao repetirmos codigo
# o mesmo poderia ser feito para a criaçao das matrizes